# Analysis of US Data Science Salary Survey

### Dowloaded from https://ai-jobs.net/salaries/download/

#### Guiding task: use dataset features to predict an individual's salary.

## 1. Import and read data

In [1]:
#global imports
import pandas as pd
import numpy as np
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

#display all dataset columns
pd.set_option('display.max_columns', 500)

df = pd.read_csv('salaries.csv')
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Data Scientist,149800,USD,149800,US,0,US,M
1,2024,MI,FT,Data Scientist,70800,USD,70800,US,0,US,M
2,2024,SE,FT,Data Scientist,220000,USD,220000,US,0,US,M
3,2024,SE,FT,Data Scientist,137500,USD,137500,US,0,US,M
4,2024,SE,FT,Data Scientist,144300,USD,144300,US,100,US,M
...,...,...,...,...,...,...,...,...,...,...,...
19494,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
19495,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
19496,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
19497,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


## 2. Prepare dataset
### I am only interested in US salaries, so we will drop the appropriate records and columns.

In [2]:
#check for empty
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

#### woah! this dataset is almost usable out-the-box.

In [3]:
#check column types
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [4]:
#keep only the records of workers who live in the US, receive their salary in USD, and whose employer is US-based
df = df[(df['employee_residence'] == 'US') & (df['salary_currency'] == 'USD') & (df['company_location'] == 'US')]
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Data Scientist,149800,USD,149800,US,0,US,M
1,2024,MI,FT,Data Scientist,70800,USD,70800,US,0,US,M
2,2024,SE,FT,Data Scientist,220000,USD,220000,US,0,US,M
3,2024,SE,FT,Data Scientist,137500,USD,137500,US,0,US,M
4,2024,SE,FT,Data Scientist,144300,USD,144300,US,100,US,M
...,...,...,...,...,...,...,...,...,...,...,...
19493,2021,SE,FT,Data Specialist,165000,USD,165000,US,100,US,L
19494,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
19495,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
19496,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S


In [5]:
# now, drop the columns that were only relevant for foreign records
df = df.drop(['salary_currency', 'salary_in_usd', 'employee_residence', 'company_location'], axis=1)
df

,work_year,experience_level,employment_type,job_title,salary,remote_ratio,company_size
0,2024,MI,FT,Data Scientist,149800,0,M
1,2024,MI,FT,Data Scientist,70800,0,M
2,2024,SE,FT,Data Scientist,220000,0,M
3,2024,SE,FT,Data Scientist,137500,0,M
4,2024,SE,FT,Data Scientist,144300,100,M
...,...,...,...,...,...,...,...
19493,2021,SE,FT,Data Specialist,165000,100,L
19494,2020,SE,FT,Data Scientist,412000,100,L
19495,2021,MI,FT,Principal Data Scientist,151000,100,L
19496,2020,EN,FT,Data Scientist,105000,100,S


### With the relevant features selected, the salary data in its current state is still not viable for comparison.
### We must adjust salaries for inflation.
#### To do that, we will evaluate the range of work years in our dataset as a parameter for scraping past inflation data 

In [6]:
# first, evaluate the earliest year in work_year
min_year = df['work_year'].min()
min_year

2020

In [7]:
# 2020 is as far back as we will have to go to find inflation data.
# find cumulative inflation rates for every year since 2024 (non-inlcusive)

## 2.1 Scraping cumulative inflation rate with Selenium

In [8]:
# set up the Selenium webdriver
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

# load the BLS page that calculates cumulative inflation rates
driver.get('https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1&year1=202001&year2=202401')

In [9]:
# implement for loop to calculate each year's cumulative inflation rate into 2024
# note: elements are fetched in every iteration because it threw error that elements became stale

cumulative_rates = {}

for i in range((2024 - min_year)):
    # enter $1
    init_dollar = driver.find_element(By.NAME, 'cost1')    
    init_dollar.clear()
    init_dollar.send_keys("1")
    init_dollar.send_keys(Keys.RETURN)
    
    # assume initial month is always January
    init_month = driver.find_element(By.ID, 'year1-month')
    dropdown = Select(init_month)
    dropdown.select_by_visible_text('January')
    
    # fill in the initial year
    init_yr = driver.find_element(By.ID, 'year1-year')
    dropdown = Select(init_yr)
    current_yr = min_year + i
    dropdown.select_by_visible_text(str(current_yr))
    
    #assume end month is always January
    present_month = driver.find_element(By.ID, 'year2-month')
    dropdown = Select(present_month)
    dropdown.select_by_visible_text('January')
    
    # fill in the present year
    present_yr = driver.find_element(By.ID, 'year2-year')
    dropdown = Select(present_yr)
    year_to_select = 2024
    dropdown.select_by_visible_text(str(year_to_select))
    
    #click the "calculate" button
    calculate_button = driver.find_element(By.ID, 'submit')
    calculate_button.click()
    
    #add cumulative interest rate to dictionary
    present_dollar = driver.find_element(By.ID, 'answer')
    r = float(present_dollar.text.strip('"').replace('$',''))
    cumulative_rates[current_yr] = r

driver.quit()
    
cumulative_rates

{2020: 1.2, 2021: 1.18, 2022: 1.1, 2023: 1.03}

### With our inflation data, we can compound past cash flows (salaries collected before 2024) into 2024.
#### Our formula for compounding is as follows, where PV is df['work_year']
$$FV_{2024} = PV \times (1 + r)^{n}$$

In [10]:
# use cumulative inflation rates to calculate adjusted salaries

# function that adjusts salary for a single row
def calculateAdjustedSalary(row):
    # get parameters for compounding formula, so PV=salary and work_year maps to the cumulative rates dictionary
    work_year = row['work_year']
    salary = row['salary']
    
    if work_year in cumulative_rates:
        r = cumulative_rates[work_year]
        salary *= r
        
    return salary

# Apply the function to each row in the DataFrame
df['inflation_adjusted_salaries'] = df.apply(calculateAdjustedSalary, axis=1).astype(int)
df

,work_year,experience_level,employment_type,job_title,salary,remote_ratio,company_size,inflation_adjusted_salaries
0,2024,MI,FT,Data Scientist,149800,0,M,149800
1,2024,MI,FT,Data Scientist,70800,0,M,70800
2,2024,SE,FT,Data Scientist,220000,0,M,220000
3,2024,SE,FT,Data Scientist,137500,0,M,137500
4,2024,SE,FT,Data Scientist,144300,100,M,144300
...,...,...,...,...,...,...,...,...
19493,2021,SE,FT,Data Specialist,165000,100,L,194700
19494,2020,SE,FT,Data Scientist,412000,100,L,494400
19495,2021,MI,FT,Principal Data Scientist,151000,100,L,178180
19496,2020,EN,FT,Data Scientist,105000,100,S,126000


In [11]:
# move inflation_adjusted_salaries to the first row
cols = ['inflation_adjusted_salaries'] + [col for col in df.columns if col != 'inflation_adjusted_salaries']
df = df[cols]

# now drop salary column
df = df.drop(columns='salary')
df

,inflation_adjusted_salaries,work_year,experience_level,employment_type,job_title,remote_ratio,company_size
0,149800,2024,MI,FT,Data Scientist,0,M
1,70800,2024,MI,FT,Data Scientist,0,M
2,220000,2024,SE,FT,Data Scientist,0,M
3,137500,2024,SE,FT,Data Scientist,0,M
4,144300,2024,SE,FT,Data Scientist,100,M
...,...,...,...,...,...,...,...
19493,194700,2021,SE,FT,Data Specialist,100,L
19494,494400,2020,SE,FT,Data Scientist,100,L
19495,178180,2021,MI,FT,Principal Data Scientist,100,L
19496,126000,2020,EN,FT,Data Scientist,100,S


## 2.2 Use OneHotEncoder on categorical data?

In [12]:
# evaluating potential dimensionality of categorical columns
print(df['experience_level'].value_counts())
print(df['employment_type'].value_counts())
print(df['job_title'].value_counts())
print(df['company_size'].value_counts())

experience_level
SE    11264
MI     4213
EN     1225
EX      490
Name: count, dtype: int64
employment_type
FT    17151
PT       23
CT       18
Name: count, dtype: int64
job_title
Data Scientist                   4052
Data Engineer                    3538
Data Analyst                     2484
Machine Learning Engineer        1947
Research Scientist                763
                                 ... 
Data Analytics Associate            1
Big Data Developer                  1
Marketing Data Scientist            1
Quantitative Research Analyst       1
Principal Data Analyst              1
Name: count, Length: 122, dtype: int64
company_size
M    16275
L      861
S       56
Name: count, dtype: int64


#### Experience level is a known factor in increasing salary, so we can maintain its ordinality in the dataset.

In [13]:
experience_mapping = {'EN': 0, 'MI': 1, 'SE': 2, 'EX':3}

# encode each level of experience with increasing ordinality
df['experience_level'] = df['experience_level'].map(experience_mapping).astype(int)
# experience level is now denoted from 0-3
df

,inflation_adjusted_salaries,work_year,experience_level,employment_type,job_title,remote_ratio,company_size
0,149800,2024,1,FT,Data Scientist,0,M
1,70800,2024,1,FT,Data Scientist,0,M
2,220000,2024,2,FT,Data Scientist,0,M
3,137500,2024,2,FT,Data Scientist,0,M
4,144300,2024,2,FT,Data Scientist,100,M
...,...,...,...,...,...,...,...
19493,194700,2021,2,FT,Data Specialist,100,L
19494,494400,2020,2,FT,Data Scientist,100,L
19495,178180,2021,1,FT,Principal Data Scientist,100,L
19496,126000,2020,0,FT,Data Scientist,100,S


#### Let's assume that there is no inherent hierarchy for company size, or whether an employee is remote, hybrid, or in-office.
#### Returning to the job title category, let's evaluate it more closely so we can avoid unnecessary dimensionality with this data feature.

In [14]:
df['job_title'].value_counts().head(50)

job_title
Data Scientist                              4052
Data Engineer                               3538
Data Analyst                                2484
Machine Learning Engineer                   1947
Research Scientist                           763
Applied Scientist                            547
Data Architect                               459
Analytics Engineer                           415
Research Engineer                            379
Business Intelligence Engineer               266
Data Manager                                 224
Business Intelligence Analyst                206
Business Intelligence                        142
AI Engineer                                  139
Research Analyst                             133
Machine Learning Scientist                   129
Data Specialist                               89
BI Developer                                  85
Data Analytics Manager                        71
Business Intelligence Developer               60
Data Model

In [15]:
# one-hot encoding


## 3. EDA